In [1]:
import numpy as np
import pandas as pd
import re
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline


C:\Users\User\Anaconda3\lib\site-packages\ipykernel\parentpoller.py:116: UserWarning: Parent poll failed.  If the frontend dies,
                the kernel may be left running.  Please let us know
                about your system (bitness, Python, etc.) at
                ipython-dev@scipy.org
  ipython-dev@scipy.org""")


In [2]:




from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont

In [3]:
train=pd.read_csv("train.csv")
test=pd.read_csv('test.csv')

PassengerId=test["PassengerId"]
train.head()
                

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
original_train=train.copy()

In [6]:
full_data=[train,test]

In [79]:
train["Has_Cabin"]=train["Cabin"].apply(lambda x:0 if type(x)==float else 1)
test["Has_Cabin"]=test["Cabin"].apply(lambda x:0 if type(x)==float else 1)

KeyError: 'Cabin'

In [16]:
train["Has_Cabin"].head()

0    0
1    1
2    0
3    1
4    0
Name: Has_Cabin, dtype: int64

In [17]:
for dataset in full_data:
    dataset['FamilySize']=dataset['SibSp']+dataset['Parch']+1
    
for datset in full_data:
    dataset['IsAlone']=0
    dataset.loc[dataset['FamilySize']==1, 'IsAlone']=1

In [26]:
age_avg

29.69911764705882

In [44]:
np.isnan(dataset['Age']).count()

418

In [47]:
for dataset in full_data:
    dataset['Embarked']=dataset['Embarked'].fillna("S")
    
for dataset in full_data:
    dataset['Fare']=dataset['Fare'].fillna(train['Fare'].median())
    
    

for dataset in full_data:
    dataset['Age']=dataset['Age'].fillna(train['Age'].mean())
    

In [49]:
for dataset in full_data:
    dataset['Sex']=dataset["Sex"].map({"female":0, "male":1}).astype(int)
    dataset["Embarked"]=dataset["Embarked"].map({"S":0,"C":1,"Q":2}).astype(int)
    dataset.loc[dataset["Fare"]<=7.91,"Fare"]
    dataset.loc[(dataset["Fare"]>7.91) & (dataset["Fare"]<=14.454),"fare"]=1
    dataset.loc[(dataset["Fare"]>14.454) & (dataset["fare"]<=31),"Fare"]=2
    dataset.loc[dataset["Fare"]>31,"Fare"]=3
    dataset["Fare"]=dataset["Fare"].astype(int)
    
    dataset.loc[dataset['Age']<=16,"Age"]=0
    dataset.loc[(dataset['Age']>16) & (dataset['Age']<=32), 'Age']=1
    dataset.loc[(dataset['Age']>32) & (dataset['Age']<=48), 'Age']=2
    dataset.loc[(dataset['Age']>48) & (dataset['Age']<=64), 'Age']=3
    dataset.loc[dataset["Age"]>64,"Age"];
    
    
    
  

In [16]:
drop_elements=["PassengerId",'Name','Ticket',"Cabin","SibSp"]
train1=train.drop(drop_elements,axis=1)
test1=test.drop(drop_elements,axis=1)

KeyError: "['PassengerId' 'Name' 'Ticket' 'Cabin' 'SibSp'] not found in axis"

In [74]:
train.head(10)

,Survived,Pclass,Sex,Age,Parch,Fare,Embarked,Has_Cabin,FamilySize,fare
0,0,3,1,1,0,7,0,0,2,NaN
1,1,1,0,2,0,3,1,1,2,NaN
2,1,3,0,1,0,7,0,0,1,1.0
3,1,1,0,2,0,3,0,1,2,NaN
4,0,3,1,2,0,8,0,0,1,1.0
5,0,3,1,1,0,8,2,0,1,1.0
6,0,1,1,3,0,3,0,1,1,NaN
7,0,3,1,0,1,21,0,0,5,NaN
8,1,3,0,1,2,11,0,0,3,1.0
9,1,2,0,0,0,30,1,0,2,NaN


In [8]:
def get_gini_impurity(survived_count, total_count):
    survival_prob=survived_count/total_count
    not_survival_prob=(1-survival_prob)
    random_observation_survived_prob=survival_prob
    random_observation_not_survived_prob=(1-random_observation_survived_prob)
    mislabelling_survived_prob=not_survival_prob*random_observation_survived_prob
    mislabelling_not_survived_prob=survival_prob*random_observation_not_survived_prob
    gini_impurity=mislabelling_survived_prob+mislabelling_not_survived_prob
    return gini_impurity


In [9]:
gini_impurity_starting_node=get_gini_impurity(342,891)
gini_impurity_starting_node

0.47301295786144265

In [10]:
gini_impurity_men=get_gini_impurity(109,577)
gini_impurity_men

0.3064437162277843

In [11]:
gini_impurity_women=get_gini_impurity(233,314)
gini_impurity_women

0.3828350034484158

In [12]:
men_weight=577/891
women_weight=314/891
weighted_gini_impurity_sex_split=(gini_impurity_men*men_weight)+(gini_impurity_women*women_weight)

sex_gini_decrease=weighted_gini_impurity_sex_split-gini_impurity_starting_node
sex_gini_decrease

-0.13964795747285214

In [13]:
gini_impurity_title_1=get_gini_impurity(81,517)
gini_impurity_title_1

0.26425329886377663

In [14]:
gini_impurity_title_others=get_gini_impurity(261,374)
gini_impurity_title_others

0.42170207898424317

In [15]:
cv=KFold(n_splits=10)
accuracies=list()
max_attributes=len(list(test))
depth_range=range(1,max_attributes+1)


for depth in depth_range:
    fold_accuracy=[]
    tree_model=tree.DecisionTreeClassifier(max_depth=depth)
    for train_fold, valid_fold in cv.split(train):
        f_train=train.loc[train_fold]
        f_valid=train.loc[valid_fold]
        
        model=tree_model.fit(X=f_train.drop(['Survived'],axis=1),
                             y=f_train["Survived"])
        valid_acc=model.score(X=f_valid.drop(['Survived'],axis=1),
                             y=f_valid['Survived'])
        fold_accuracy.append(valid_acc)

        
    avg=sum(fold_accuracy)/len(fold_accuracy)
    accuracies.append(avg)
            
            
            
            
df=pd.DataFrame({"Max Depth":depth_range, "Average Accuracy": accuracies})
df=df[["Max Depth", "Average Accuracy"]]
print(df.to_string(index=False))            
                             
















                        

ValueError: could not convert string to float: 'male'